### raw data
* word embedding: glove
* doc text: ./data/IMDB.txt

### dataset
1. IMDB
2. CNNNews
3. [PubMed](https://github.com/LIAAD/KeywordExtractor-Datasets/blob/master/datasets/PubMed.zip)

### preprocess
1. filter too frequent and less frequent words
2. stemming
3. document vector aggregation

### model
1. TopK
2. Sklearn
3. Our model

### evaluation
1. F1
2. NDCG

In [1]:
import os
from collections import defaultdict
import math
import numpy as np 
import random
import re
import torch
import torch.nn as nn
from itertools import cycle
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from tqdm.auto import tqdm

# Used to get the data
from sklearn.metrics import ndcg_score

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
nltk.download('stopwords')

import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('Agg')

seed = 33
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chrisliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocess config

In [2]:
config = {}

config["dataset"] = "CNN" # "IMDB" "CNN", "PubMed"
config["n_document"] = 10000
config["normalize_word_embedding"] = True
config["min_word_freq_threshold"] = 20
config["topk_word_freq_threshold"] = 100
config["document_vector_agg_weight"] = 'IDF' # ['mean', 'IDF', 'uniform', 'gaussian', 'exponential', 'pmi']
config["document_vector_weight_normalize"] = False # weighted sum or mean, True for sum, False for mean 
config["select_topk_TFIDF"] = None # ignore
config["embedding_file"] = "../data/glove.6B.100d.txt"
config["topk"] = [10, 30, 50]


In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    return True

In [4]:
def load_word2emb(embedding_file):
    word2embedding = dict()
    word_dim = int(re.findall(r".(\d+)d", embedding_file)[0])

    with open(embedding_file, "r") as f:
        for line in tqdm(f):
            line = line.strip().split()
            word = line[0]
            embedding = list(map(float, line[1:]))
            word2embedding[word] = np.array(embedding)

    print("Number of words:%d" % len(word2embedding))

    return word2embedding

word2embedding = load_word2emb(config["embedding_file"])

0it [00:00, ?it/s]

Number of words:400000


In [5]:
def normalize_wordemb(word2embedding):
    # Every word emb should have norm 1
    
    word_emb = []
    word_list = []
    for word, emb in word2embedding.items():
        word_list.append(word)
        word_emb.append(emb)

    word_emb = np.array(word_emb)

    for i in range(len(word_emb)):
        norm = np.linalg.norm(word_emb[i])
        word_emb[i] = word_emb[i] / norm

    for word, emb in tqdm(zip(word_list, word_emb)):
        word2embedding[word] = emb
    return word2embedding

if config["normalize_word_embedding"]:
    normalize_wordemb(word2embedding)

0it [00:00, ?it/s]

In [6]:
class Vocabulary:
    def __init__(self, word2embedding, config):
        # The low frequency words will be assigned as <UNK> token
        self.itos = {0: "<UNK>"}
        self.stoi = {"<UNK>": 0}
        
        self.word2embedding = word2embedding
        self.config = config

        self.word_freq_in_corpus = defaultdict(int)
        self.IDF = {}
        self.ps = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        
        self.word_dim = len(word2embedding['the'])
    def __len__(self):
        return len(self.itos)

    def tokenizer_eng(self, text):
        text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
        text = text.strip().split()
        
        return [self.ps.stem(w) for w in text if w.lower() not in self.stop_words]
    
    def read_raw(self):        
        if self.config["dataset"] == 'IMDB':
            data_file_path = '../data/IMDB.txt'
        elif self.config["dataset"] == 'CNN':
            data_file_path = '../data/CNN.txt'
        elif self.config["dataset"] == 'PubMed':
            data_file_path = '../data/PubMed.txt'
        
        # raw documents
        self.raw_documents = []
        with open(data_file_path,'r',encoding='utf-8') as f:
            for line in tqdm(f, desc="Loading documents"):
                self.raw_documents.append(line.strip("\n"))
                
        return self.raw_documents
    
    def build_vocabulary(self):
        sentence_list = self.raw_documents
        
        self.doc_freq = defaultdict(int) # # of document a word appear
        self.document_num = len(sentence_list)
        self.word_vectors = [[0]*self.word_dim] # unknown word emb
        
        for sentence in tqdm(sentence_list, desc="Preprocessing documents"):
            # for doc_freq
            document_words = set()
            
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.word2embedding:
                    continue
                    
                # calculate word freq
                self.word_freq_in_corpus[word] += 1
                document_words.add(word)
                
            for word in document_words:
                self.doc_freq[word] += 1
        
        # calculate IDF
        print('doc num', self.document_num)
        for word, freq in self.doc_freq.items():
            self.IDF[word] = math.log(self.document_num / (freq+1))
        
        # delete less freq words:
        delete_words = []
        for word, v in self.word_freq_in_corpus.items():
            if v < self.config["min_word_freq_threshold"]:
                delete_words.append(word)     
        for word in delete_words:
            del self.IDF[word]    
            del self.word_freq_in_corpus[word]    
        
        # delete too freq words
        print('eliminate freq words')
        IDF = [(word, freq) for word, freq in self.IDF.items()]
        IDF.sort(key=lambda x: x[1])

        for i in range(self.config["topk_word_freq_threshold"]):
            print(word)
            word = IDF[i][0]
            del self.IDF[word]
            del self.word_freq_in_corpus[word]
        
        # construct word_vectors
        idx = 1
        for word in self.word_freq_in_corpus:
            self.word_vectors.append(self.word2embedding[word])
            self.stoi[word] = idx
            self.itos[idx] = word
            idx += 1
            
    def init_word_weight(self,sentence_list, agg):
        if agg == 'mean':
            self.word_weight = {word: 1 for word in self.IDF.keys()}
        elif agg == 'IDF':
            self.word_weight = self.IDF
        elif agg == 'uniform':
            self.word_weight = {word: np.random.uniform(low=0.0, high=1.0) for word in self.IDF.keys()}
        elif agg == 'gaussian':
            mu, sigma = 10, 1 # mean and standard deviation
            self.word_weight = {word: np.random.normal(mu, sigma) for word in self.IDF.keys()}
        elif agg == 'exponential':
            self.word_weight = {word: np.random.exponential(scale=1.0) for word in self.IDF.keys()}
        elif agg == 'pmi':
            trigram_measures = BigramAssocMeasures()
            self.word_weight = defaultdict(int)
            corpus = []

            for text in tqdm(sentence_list):
                corpus.extend(text.split())

            finder = BigramCollocationFinder.from_words(corpus)
            for pmi_score in finder.score_ngrams(trigram_measures.pmi):
                pair, score = pmi_score
                self.word_weight[pair[0]] += score
                self.word_weight[pair[1]] += score
                
    def calculate_document_vector(self):
        # Return
        # document_vectors: weighted sum of word emb
        # document_answers_idx: doc to word index list
        # document_answers_wsum: word weight summation, e.g. total TFIDF score of a doc
        
        document_vectors = [] 
        document_answers = []
        document_answers_wsum = []
        
        sentence_list = self.raw_documents
        agg = self.config["document_vector_agg_weight"]
        n_document = self.config["n_document"]
        select_topk_TFIDF = self.config["select_topk_TFIDF"]
        
        self.init_word_weight(sentence_list, agg)
        for sentence in tqdm(sentence_list[:min(n_document, len(sentence_list))], desc="calculate document vectors"):
            document_vector = np.zeros(len(self.word_vectors[0]))
            select_words = []
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.stoi:
                    continue
                else:
                    select_words.append(word)

            # select topk TDIDF
            if select_topk_TFIDF is not None:
                doc_TFIDF = defaultdict(float)
                for word in select_words:    
                    doc_TFIDF[word] += self.IDF[word]

                doc_TFIDF_l = [(word, TFIDF) for word, TFIDF in doc_TFIDF.items()]
                doc_TFIDF_l.sort(key=lambda x:x[1], reverse=True)
                
                select_topk_words = set(list(map(lambda x:x[0], doc_TFIDF_l[:select_topk_TFIDF])))
                select_words = [word for word in select_words if word in select_topk_words]
            else:
                pass
            
            total_weight = 0
            # aggregate to doc vectors
            for word in select_words:
                document_vector += np.array(self.word2embedding[word]) * self.word_weight[word]
                total_weight += self.word_weight[word]
                
            if len(select_words) == 0:
                print('error', sentence)
                continue
            else:
                if self.config["document_vector_weight_normalize"]:
                    document_vector /= total_weight
            
            document_vectors.append(document_vector)
            document_answers.append(select_words)
            document_answers_wsum.append(total_weight)
        
        # get answers
        document_answers_idx = []    
        for ans in document_answers:
            ans_idx = []
            for token in ans:
                if token in self.stoi:
                    ans_idx.append(self.stoi[token])                    
            document_answers_idx.append(ans_idx)
        
        self.document_vectors = document_vectors
        self.document_answers_idx = document_answers_idx
        self.document_answers_wsum = document_answers_wsum
        
        return document_vectors, document_answers_idx, document_answers_wsum
        
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]
    
    def check_docemb(self):
        word_vectors = np.array(self.word_vectors)
        pred = np.zeros(word_vectors.shape[1])
        cnt = 0

        for word_idx in self.document_answers_idx[0]:
            pred += word_vectors[word_idx] * self.word_weight[self.itos[word_idx]]
            cnt += self.word_weight[self.itos[word_idx]]
        
        if self.config["document_vector_weight_normalize"]:
            pred /= cnt
        assert np.sum(self.document_vectors[0]) - np.sum(pred) == 0

In [7]:
def build_vocab(config, word2embedding):
    # build vocabulary
    vocab = Vocabulary(word2embedding, config)
    vocab.read_raw()
    vocab.build_vocabulary()
    vocab_size = len(vocab)
    # get doc emb
    vocab.calculate_document_vector()
    vocab.check_docemb()
    
    return vocab

vocab = build_vocab(config, word2embedding)

Loading documents: 0it [00:00, ?it/s]

Preprocessing documents:   0%|          | 0/19026 [00:00<?, ?it/s]

doc num 19026
eliminate freq words
paroxysm
subject
line
organ
write
univers
one
would
use
like
get
know
dont
think
time
make
also
say
go
im
could
want
new
work
good
well
way
need
look
even
anyon
thing
see
tri
thank
much
year
world
system
right
problem
may
take
mani
two
first
seem
question
pleas
1
state
us
come
2
post
help
call
usa
point
sinc
find
read
still
back
mean
ive
give
email
sure
differ
might
run
cant
reason
last
day
interest
case
let
person
said
never
start
doesnt
tell
better
ask
got
without
follow
part
lot
3
number
put
fact
gener
inform
actual
that


calculate document vectors:   0%|          | 0/10000 [00:00<?, ?it/s]

error |> 
error 
error 
error Mikael Fredriksson
error 
error -------------------------------------------------
error email: mikael_fredriksson@macexchange.se
error 
error FIDO 2:203/211
error  
error  


In [8]:
print("Finish building dataset!")
print(f"Number of documents:{len(vocab.raw_documents)}")
print(f"Number of words:{len(vocab)}")

l = list(map(len, vocab.document_answers_idx))
print("Average length of document:", np.mean(l))

Finish building dataset!
Number of documents:19026
Number of words:7602
Average length of document: 86.49083992391631


In [9]:
word_vectors = np.array(vocab.word_vectors)
print(word_vectors.shape)

document_vectors = np.array(vocab.document_vectors)
print(document_vectors.shape)

document_answers_wsum = np.array(vocab.document_answers_wsum).reshape(-1, 1)
print(document_answers_wsum.shape)

# create weight_ans
document_answers_idx = vocab.document_answers_idx

# random shuffle
shuffle_idx = list(range(len(document_vectors)))
random.Random(seed).shuffle(shuffle_idx)

document_vectors = document_vectors[shuffle_idx]
document_answers_wsum = document_answers_wsum[shuffle_idx]
document_answers_idx = [document_answers_idx[idx] for idx in shuffle_idx]

(7602, 100)
(9989, 100)
(9989, 1)


In [10]:
# onthot_ans: word freq matrix
# weight_ans: TFIDF matrix

onehot_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
weight_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
print(weight_ans.shape)

for i in tqdm(range(len(document_answers_idx))):
    for word_idx in document_answers_idx[i]:
        weight_ans[i, word_idx] += vocab.word_weight[vocab.itos[word_idx]]
        onehot_ans[i, word_idx] += 1

(9989, 7602)


  0%|          | 0/9989 [00:00<?, ?it/s]

## Results

In [11]:
final_results = []
select_columns = ['model']
for topk in config["topk"]:
    select_columns.append('F1@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('ndcg@{}'.format(topk))
select_columns.append('ndcg@all')
select_columns

['model',
 'F1@10',
 'F1@30',
 'F1@50',
 'ndcg@10',
 'ndcg@30',
 'ndcg@50',
 'ndcg@all']

## setting training size

In [12]:
train_size_ratio = 1
train_size = int(len(document_answers_idx) * train_size_ratio)
train_size

9989

## Top K freq word

In [13]:
topk_results = {}

In [14]:
test_ans = document_answers_idx[:train_size]

In [15]:
word_freq = [(word, freq) for word, freq in vocab.word_freq_in_corpus.items()]
word_freq.sort(key=lambda x:x[1], reverse=True)
word_freq[:10]

[('x', 6539),
 ('god', 5208),
 ('file', 4918),
 ('0', 4520),
 ('window', 4444),
 ('program', 4201),
 ('drive', 3633),
 ('4', 3528),
 ('game', 3474),
 ('govern', 3268)]

In [16]:
def topk_word_evaluation(k=50):
    topk_word = [word for (word, freq) in word_freq[:k]]

    pr, re = [], []
    for ans in tqdm(test_ans):
        ans = set(ans)
        ans = [vocab.itos[a] for a in ans]

        hit = []
        for word in ans:
            if word in topk_word:
                hit.append(word)

        precision = len(hit) / k
        recall = len(hit) / len(ans)
        pr.append(precision)
        re.append(recall)

    pr = np.mean(pr)
    re = np.mean(re)
    f1 = 2 * pr * re / (pr + re) if (pr + re) != 0 else 0
    print('top {} word'.format(k))
    print('percision', np.mean(pr))
    print('recall', np.mean(re))
    print('F1', f1)
    return f1


for topk in config['topk']:
    topk_results["F1@{}".format(topk)] = topk_word_evaluation(k=topk)


  0%|          | 0/9989 [00:00<?, ?it/s]

top 10 word
percision 0.07090799879867855
recall 0.016414023763480924
F1 0.02665732064265653


  0%|          | 0/9989 [00:00<?, ?it/s]

top 30 word
percision 0.07792238128608137
recall 0.04873623777882011
F1 0.05996660520519939


  0%|          | 0/9989 [00:00<?, ?it/s]

top 50 word
percision 0.07793572930223247
recall 0.07962461018581644
F1 0.07877111823192375


In [17]:
def topk_word_evaluation_NDCG(k=50):
    freq_word =[word for (word, freq) in word_freq]
    freq_word_idx = [vocab.stoi[word] for word in freq_word if word in vocab.stoi]
    
    scores = np.zeros(len(vocab.word_vectors))
    for rank, idx in enumerate(freq_word_idx):
        scores[idx] = len(vocab.word_vectors) - rank
    
    NDCGs = []
    
    for ans in tqdm(test_ans):
        weight_ans = np.zeros(len(vocab.word_vectors))
        
        for word_idx in ans:
            if word_idx == 0:
                continue
            word = vocab.itos[word_idx]
            weight_ans[word_idx] += vocab.IDF[word]

        NDCG_score = ndcg_score(weight_ans.reshape(1,-1), scores.reshape(1,-1), k=k)
        NDCGs.append(NDCG_score)

    print('top {} NDCG:{}'.format(k, np.mean(NDCGs)))
    
    return np.mean(NDCGs)


for topk in config['topk']:
    topk_results["ndcg@{}".format(topk)] = topk_word_evaluation_NDCG(k=topk)
    
topk_results["ndcg@all"] = topk_word_evaluation_NDCG(k=None)


  0%|          | 0/9989 [00:00<?, ?it/s]

top 10 NDCG:0.02843140685579532


  0%|          | 0/9989 [00:00<?, ?it/s]

top 30 NDCG:0.03790739702719704


  0%|          | 0/9989 [00:00<?, ?it/s]

top 50 NDCG:0.046074423314069815


  0%|          | 0/9989 [00:00<?, ?it/s]

top None NDCG:0.31543624801354186


In [18]:
topk_results["model"] = "topk"
final_results.append(pd.Series(topk_results))

## Sklearn

In [19]:
from sklearn.linear_model import LinearRegression, Lasso

In [20]:
print(document_vectors.shape)
print(weight_ans.shape)
print(word_vectors.shape)

(9989, 100)
(9989, 7602)
(7602, 100)


In [21]:
def evaluate_sklearn(pred, ans):
    results = {}
        
    one_hot_ans = np.arange(ans.shape[0])[ans > 0]
    
    for topk in config["topk"]:
        one_hot_pred = np.argsort(pred)[-topk:]
        hit = np.intersect1d(one_hot_pred, one_hot_ans)
        percision = len(hit) / topk
        recall = len(hit) / len(one_hot_ans)
        f1 = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
        
        results['F1@{}'.format(topk)] = f1
        
    ans = ans.reshape(1, -1)
    pred = pred.reshape(1, -1)
    for topk in config["topk"]:
        results['ndcg@{}'.format(topk)] = ndcg_score(ans, pred, k=topk)

    results['ndcg@all'] = (ndcg_score(ans, pred, k=None))
    
    return results

### linear regression

In [22]:
# results = []

# for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
#     x = word_vectors.T
#     y = doc_emb
    
#     ans = weight_ans[doc_id]
#     model = LinearRegression(fit_intercept=False).fit(x, y)
#     r2 = model.score(x, y)

#     res = evaluate_sklearn(model.coef_, ans)
#     results.append(res)

In [23]:
# results = pd.DataFrame(results).mean()
# results['model'] = 'sk-linear-regression'
# final_results.append(results)
# results

### lasso

In [24]:
# results = []
# sk_lasso_epoch = 10000

# for doc_id, doc_emb in enumerate(tqdm(document_vectors[:train_size])):
#     x = word_vectors.T
#     y = doc_emb
    
#     ans = weight_ans[doc_id]
#     model = Lasso(positive=True, fit_intercept=False, alpha=0.0001, max_iter=sk_lasso_epoch, tol=0).fit(x, y)
#     r2 = model.score(x, y)

#     res = evaluate_sklearn(model.coef_, ans)
#     results.append(res)

In [25]:
# results = pd.DataFrame(results).mean()
# results['model'] = 'sk-lasso'
# final_results.append(results)
# results

## Our Model

In [26]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [27]:
class Custom_Lasso_Dataset(Dataset):
    def __init__(self, 
                 doc_vectors,
                 doc_w_sum,
                 weight_ans
                 ):
        self.doc_vectors = torch.FloatTensor(doc_vectors)
        self.doc_w_sum = torch.FloatTensor(doc_w_sum)
        self.weight_ans = weight_ans
        assert len(doc_vectors) == len(doc_w_sum)
        
    def __getitem__(self, idx):
                
        return self.doc_vectors[idx], self.doc_w_sum[idx], idx

    def __len__(self):
        return len(self.doc_vectors)


In [28]:
class LR(nn.Module):
    """
    Input shape: (N, 3, 64, 64)
    Output shape: (N, )
    """
    def __init__(self, num_doc, num_words):
        super(LR, self).__init__()
        weight = torch.zeros(num_doc, num_words).to(device)
        self.emb = torch.nn.Embedding.from_pretrained(weight, freeze=False)
        
    def forward(self, doc_ids, word_vectors):
        return self.emb(doc_ids) @ word_vectors

In [29]:
def evaluate_NDCG(model, train_loader):
    results = {}
    model.eval()
    
    scores = np.array(model.emb.cpu().weight.data)
    model.emb.to(device)
    true_relevance = train_loader.dataset.weight_ans

    # F1
    F1s = []
    for i in range(true_relevance.shape[0]):
        one_hot_ans = np.arange(true_relevance.shape[1])[true_relevance[i] > 0]
        pred = scores[i]
        
        F1 = []
        for topk in config["topk"]:
            one_hot_pred = np.argsort(pred)[-topk:]
            
            hit = np.intersect1d(one_hot_pred, one_hot_ans)
            percision = len(hit) / topk
            recall = len(hit) / len(one_hot_ans)
            
            ans = 2 * percision * recall / (percision + recall) if (percision + recall) > 0 else 0
            F1.append(ans)
        F1s.append(F1)
        
    F1s = np.mean(F1s, axis=0)
    
    for i, topk in enumerate(config["topk"]):
        results['F1@{}'.format(topk)] = F1s[i]

    # NDCG
    for topk in config["topk"]:
        results['ndcg@{}'.format(topk)] = ndcg_score(true_relevance, scores, k=topk)
    results['ndcg@all'] = ndcg_score(true_relevance, scores, k=None)
    
    return results

In [30]:
batch_size = 100
print('document num', train_size)

train_dataset = Custom_Lasso_Dataset(document_vectors[:train_size], document_answers_wsum[:train_size], weight_ans[:train_size])
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

document num 9989


## start training

In [31]:
# # setting
# lr = 0.5
# momentum = 0.99
# weight_decay = 0
# nesterov = False # True

# n_epoch = 50000

# w_sum_reg = 1e-3
# w_sum_reg_mul = 0.9
# w_clip_value = 0

# L1 = 1e-6

# verbose = False
# valid_epoch = 100

# model = LR(num_doc=train_size, num_words=word_vectors.shape[0]).to(device)
# model.train()

# word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)
    
# opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
# criterion = nn.MSELoss(reduction='mean')

# results = []
# step = 0
# for epoch in tqdm(range(n_epoch)):    
#     loss_mse_his = []
#     loss_w_reg_his = []
    
#     model.train()

#     for data in train_loader:
#         doc_embs, doc_w_sum, doc_ids = data
        
#         doc_embs = doc_embs.to(device)
#         doc_w_sum = doc_w_sum.to(device)
#         doc_ids = doc_ids.to(device)
        
#         w_reg = (torch.ones(doc_embs.size(0), 1) * w_sum_reg_mul).to(device)
        
#         # MSE loss
#         pred_doc_embs = model(doc_ids, word_vectors_tensor)     
#         loss_mse = criterion(pred_doc_embs, doc_embs)

#         pred_w_sum = torch.sum(model.emb(doc_ids), axis=1).view(-1, 1)
#         loss_w_reg = criterion(pred_w_sum, w_reg)
        
#         loss_l1 = torch.sum(torch.abs(model.emb(doc_ids)))
#         loss = loss_mse + loss_w_reg * w_sum_reg + loss_l1 * L1
        
#         # Model backwarding
#         model.zero_grad()
#         loss.backward()
#         opt.step()

#         loss_mse_his.append(loss_mse.item())
#         loss_w_reg_his.append(loss_w_reg.item())

#         for p in model.parameters():
#             p.data.clamp_(w_clip_value, float('inf'))

        
#     if epoch % valid_epoch == 0:
#         res = {}
#         res['epoch'] = epoch
#         res['loss_mse'] = np.mean(loss_mse_his)
#         res['loss_w_reg'] = np.mean(loss_w_reg_his)
        
#         res_ndcg = evaluate_NDCG(model, train_loader)
#         res.update(res_ndcg)
#         results.append(res)
        
#         if verbose:
#             print()
#             for k, v in res.items():
#                 print(k, v)

In [32]:
# pd.set_option('display.max_rows', 500)
# results_df = pd.DataFrame(results).set_index('epoch')
# results_df

NameError: name 'results' is not defined

In [33]:
# results_df['model'] = 'our-lasso'
# final_results.append(results_df[select_columns].iloc[-1])

## Quality Check

In [34]:
# # select doc_id and k
# doc_id = 40
# topk = 30

# model

In [35]:
# import colored
# from colored import stylize

# word_list = vocab.itos

# gt = [word_list[word_idx] for word_idx in np.argsort(weight_ans[doc_id])[::-1][:topk]]
# pred = [word_list[word_idx] for word_idx in np.argsort(model.emb.cpu().weight.data[doc_id].numpy())[::-1][:topk]]

# print('ground truth')
# for word in gt:
#     if word in pred:
#         print(stylize(word, colored.bg("yellow")), end=' ')
#     else:
#         print(word, end=' ')

# print()
# print('\nprediction')
# for word in pred:
#     if word in gt:
#         print(stylize(word, colored.bg("yellow")), end=' ')
#     else:
#         print(word, end=' ')


In [36]:
# # raw document
# print()
# ps = PorterStemmer()
    
# for word in vocab.raw_documents[doc_id].split():
#     word_stem = ps.stem(word).lower()

#     if word_stem in gt:
#         if word_stem in pred:
#             print(stylize(word, colored.bg("yellow")), end=' ')
#         else:
#             print(stylize(word, colored.bg("light_gray")), end=' ')
#     else:
#         print(word, end=' ')
# # print(dataset.documents[doc_id])

In [37]:
# results = {}
   
# scores = np.array(model.emb.weight.data)[doc_id].reshape(1, -1)
# true_relevance = train_loader.dataset.weight_ans[doc_id].reshape(1, -1)

# results['ndcg@50'] = (ndcg_score(true_relevance, scores, k=50))
# results['ndcg@100'] = (ndcg_score(true_relevance, scores, k=100))
# results['ndcg@200'] = (ndcg_score(true_relevance, scores, k=200))
# results['ndcg@all'] = (ndcg_score(true_relevance, scores, k=None))

# print('This document ndcg:')
# print('ground truth length:', np.sum(weight_ans[doc_id] > 0))
# print('NDCG top50', results['ndcg@50'])
# print('NDCG top100', results['ndcg@100'])
# print('NDCG top200', results['ndcg@200'])
# print('NDCG ALL', results['ndcg@all'])


## Final results

In [38]:
is_notebook = in_notebook()

In [39]:
# final_results_df = pd.DataFrame(final_results).reset_index(drop=True)

# experiment_dir = './records/dataset-{}-n_document-{}-wdist-{}-filtertopk-{}'.format(
#                                         config['dataset'],
#                                         config['n_document'],
#                                         config["document_vector_agg_weight"],
#                                         config["topk_word_freq_threshold"])

# print('Saving to directory', experiment_dir)
# os.makedirs(experiment_dir, exist_ok=True)

In [40]:
# final_results_df.to_csv(os.path.join(experiment_dir, 'result.csv'), index=False)

# import json
# with open(os.path.join(experiment_dir, 'config.json'), 'w') as json_file:
#     json.dump(config, json_file)

In [41]:
# for feat in final_results_df.set_index('model').columns:
#     plt.bar(final_results_df['model'],
#             final_results_df[feat], 
#             width=0.5, 
#             bottom=None, 
#             align='center', 
#             color=['lightsteelblue', 
#                    'cornflowerblue', 
#                    'royalblue', 
#                    'navy'])
#     plt.title(feat)
#     plt.savefig(os.path.join(experiment_dir, '{}.png'.format(feat)))
#     plt.clf()
#     if is_notebook:
#         plt.show()

In [42]:
# print(final_results_df)

## MLP Decoder

In [43]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [44]:
class MLPDecoderDataset(Dataset):
    def __init__(self, 
                 doc_vectors,
                 weight_ans,
                 topk=50):
        self.doc_vectors = torch.FloatTensor(doc_vectors)
        self.weight_ans = torch.FloatTensor(weight_ans)
        self.weight_ans_s = torch.argsort(self.weight_ans, dim=1, descending=True)
        self.weight_ans_s[:, topk:] = -1
        
        assert len(doc_vectors) == len(weight_ans)
        
    def __getitem__(self, idx):
        return self.doc_vectors[idx], self.weight_ans[idx], self.weight_ans_s[idx]

    def __len__(self):
        return len(self.doc_vectors)

In [45]:
batch_size = 100

train_size_ratio = 0.9
train_size = int(len(document_answers_idx) * train_size_ratio)
train_size

print('train size', train_size)
print('valid size', len(document_vectors) - train_size)

train_dataset = MLPDecoderDataset(document_vectors[:train_size], weight_ans[:train_size], topk=50)
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

valid_dataset = MLPDecoderDataset(document_vectors[train_size:], weight_ans[train_size:], topk=50)
valid_loader  = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

train size 8990
valid size 999


In [46]:
class MLPDecoder(nn.Module):
    def __init__(self, doc_emb_dim, num_words, h_dim=300):
        super().__init__()
        
        self.fc1 = nn.Linear(doc_emb_dim, h_dim) 
#         self.fc2 = nn.Linear(h_dim, h_dim)
#         self.fc3 = nn.Linear(h_dim, h_dim)
        self.fc4 = nn.Linear(h_dim, num_words)
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
#         x = F.tanh(self.fc2(x))
#         x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        
        return x

In [47]:
from sklearn.metrics import f1_score
import timeit
from torchmetrics import F1
from torchmetrics.functional import retrieval_normalized_dcg

def evaluate_MLPDecoder(model, data_loader):
    results = {}
    model.eval()
    
    pred_all = []
    target_all = []
    
    # predict all data
    start = timeit.default_timer()
    for data in data_loader:
        doc_embs, target, _ = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
                
        pred = model(doc_embs)
        pred_all.append(pred)
        target_all.append(target)
        
    pred_all = torch.cat(pred_all, dim=0)
    target_all = torch.cat(target_all, dim=0)
    stop1 = timeit.default_timer()
    print('Time1: ', stop1 - start)  
    
    # F1
    for topk in config["topk"]:
        f1 = F1(top_k=topk)
        f1_score = f1(pred_all.cpu(), torch.sign(target_all).int().cpu())
        results['F1@{}'.format(topk)] = f1_score.item()
    stop2 = timeit.default_timer()
    print('Time2: ', stop2 - stop1)  
    
    # NDCG
    for topk in config["topk"]:
        ndcg_scores = []
        for i in range(pred_all.shape[0]):
            ndcg_scores.append(retrieval_normalized_dcg(pred_all[i], target_all[i], k=topk).item())     
        results['ndcg@{}'.format(topk)] = np.mean(ndcg_scores)
        
    ndcg_scores = []
    for i in range(pred_all.shape[0]):
        ndcg_scores.append(retrieval_normalized_dcg(pred_all[i], target_all[i]).item())     
    results['ndcg@all'] = np.mean(ndcg_scores)
    stop3 = timeit.default_timer()
    print('Time3: ', stop3 - stop2)      
    
    return results

In [48]:
DEFAULT_EPS = 1e-10
PADDED_Y_VALUE = -1

In [54]:
# setting
lr = 0.05
momentum = 0.
weight_decay = 0
nesterov = False # True

n_epoch = 5000
verbose = True
valid_epoch = 50

h_dim = 3000

model = MLPDecoder(doc_emb_dim=document_vectors.shape[1], num_words=len(word_vectors), h_dim=h_dim).to(device)
model.train()

word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)

opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
# opt = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.MSELoss(reduction='mean')
criterion = nn.MultiLabelMarginLoss(reduction='mean')
# criterion = nn.MultiLabelSoftMarginLoss(reduction='mean')

results = []
step = 0
clip_value = 1

for epoch in tqdm(range(n_epoch)):    
    train_loss_his = []
    valid_loss_his = []
    
    model.train()

    for data in train_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
#         loss = criterion(pred, target)
        loss = criterion(pred, target_rank)
#         loss = criterion(pred, torch.sign(target))
        
        # Model backwarding
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value)
        
        opt.step()

        train_loss_his.append(loss.item())
        
    model.eval()
    for data in valid_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
        
#         loss = criterion(pred, target)
        loss = criterion(pred, target_rank)
#         loss = criterion(pred, torch.sign(target))
        
        valid_loss_his.append(loss.item())
    
    print("Epoch", epoch, np.mean(train_loss_his), np.mean(valid_loss_his))
    
    if epoch % valid_epoch == 0:
        res = {}
        res['epoch'] = epoch
        
        train_res_ndcg = evaluate_MLPDecoder(model, train_loader)
        valid_res_ndcg = evaluate_MLPDecoder(model, valid_loader)
        
        res.update(valid_res_ndcg)
        results.append(res)
        
        if verbose:
            print()
            print('train', train_res_ndcg)
            print('valid', valid_res_ndcg)
            


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0 16.27092376285129 11.797836303710938
Time1:  0.31581695564091206
Time2:  5.738117668777704
Time3:  20.17899980954826
Time1:  0.029079269617795944
Time2:  0.6093807443976402
Time3:  2.2846240177750587

train {'F1@10': 0.022304276004433632, 'F1@30': 0.054059337824583054, 'F1@50': 0.08223669230937958, 'ndcg@10': 0.046150971230938385, 'ndcg@30': 0.06001905465219636, 'ndcg@50': 0.07714005401871453, 'ndcg@all': 0.36135738962583736}
valid {'F1@10': 0.022343382239341736, 'F1@30': 0.05248989164829254, 'F1@50': 0.07892013341188431, 'ndcg@10': 0.046766206919046885, 'ndcg@30': 0.05951047831497438, 'ndcg@50': 0.07486342325190397, 'ndcg@all': 0.3565012764584672}
Epoch 1 8.92164199617174 9.856325054168702
Epoch 2 6.442120212978787 9.010136842727661
Epoch 3 5.077125724156698 8.550235843658447
Epoch 4 4.203001186582777 8.289485311508178
Epoch 5 3.593711259629991 8.111674070358276
Epoch 6 3.1613865507973564 7.971252822875977
Epoch 7 2.8388562917709352 7.8873515129089355
Epoch 8 2.588604919115702

Epoch 138 0.30710871981249915 7.359880208969116
Epoch 139 0.30371472239494324 7.3572102069854735
Epoch 140 0.3022480378548304 7.3659289360046385
Epoch 141 0.30153957075542875 7.3633434772491455
Epoch 142 0.30019175046020086 7.367502212524414
Epoch 143 0.29837778045071495 7.370369482040405
Epoch 144 0.2964234656757779 7.365205764770508
Epoch 145 0.29498112201690674 7.368148612976074
Epoch 146 0.2935927285088433 7.362018489837647
Epoch 147 0.29078970717059244 7.370318078994751
Epoch 148 0.2906249708599514 7.367989540100098
Epoch 149 0.2885123252868652 7.3690392017364506
Epoch 150 0.2881535937388738 7.361169052124024
Time1:  0.2586024980992079
Time2:  4.051573064178228
Time3:  15.940886987373233
Time1:  0.02937052771449089
Time2:  0.42283318378031254
Time3:  1.754525724798441

train {'F1@10': 0.16021370887756348, 'F1@30': 0.39880651235580444, 'F1@50': 0.568867027759552, 'ndcg@10': 0.43356649215427434, 'ndcg@30': 0.5214489496970867, 'ndcg@50': 0.6168439858943199, 'ndcg@all': 0.692684572865

Epoch 272 0.18382354312472873 7.4549102783203125
Epoch 273 0.18319454424911075 7.454755544662476
Epoch 274 0.1823247199257215 7.452724933624268
Epoch 275 0.18207369877232446 7.466057682037354
Epoch 276 0.18207059287362629 7.460947799682617
Epoch 277 0.18161441485087076 7.454405498504639
Epoch 278 0.1816793700059255 7.46027102470398
Epoch 279 0.1805549446079466 7.464733695983886
Epoch 280 0.1810360550880432 7.469304418563842
Epoch 281 0.1796016866962115 7.467050790786743
Epoch 282 0.1791052887837092 7.471155977249145
Epoch 283 0.17918475137816536 7.46147608757019
Epoch 284 0.17797645578781765 7.459161710739136
Epoch 285 0.17749439941512213 7.463137483596801
Epoch 286 0.17736505998505486 7.469886207580567
Epoch 287 0.17658910519546933 7.468975162506103
Epoch 288 0.17569056500991184 7.470464134216309
Epoch 289 0.1768105637696054 7.47409200668335
Epoch 290 0.17539538542429606 7.477338218688965
Epoch 291 0.17525885668065813 7.469485187530518
Epoch 292 0.17476791391770044 7.472585535049438
E

Epoch 406 0.13802005391981867 7.550264883041382
Epoch 407 0.13789961561560632 7.55513710975647
Epoch 408 0.13768347021606234 7.556053400039673
Epoch 409 0.1376279557744662 7.54733452796936
Epoch 410 0.13683834175268808 7.558027458190918
Epoch 411 0.13618515125579303 7.563369369506836
Epoch 412 0.1364024424718486 7.551956129074097
Epoch 413 0.13581805965966648 7.558000946044922
Epoch 414 0.13537215929892327 7.55357928276062
Epoch 415 0.13526911611358325 7.553326511383057
Epoch 416 0.13528130261434448 7.551773738861084
Epoch 417 0.13555233139130804 7.550339984893799
Epoch 418 0.1354444960753123 7.55699405670166
Epoch 419 0.13541649762127134 7.557705736160278
Epoch 420 0.13464082520869042 7.554332828521728
Epoch 421 0.1349620213939084 7.559082412719727
Epoch 422 0.13376895462473234 7.560801887512207
Epoch 423 0.1332315791812208 7.558249759674072
Epoch 424 0.1335543279018667 7.553527402877807
Epoch 425 0.1344457829164134 7.556572103500367
Epoch 426 0.1333683422870106 7.557716274261475
Epoc

...

Epoch 4328 0.029871476689974467 8.63325548171997
Epoch 4329 0.029457109545667968 8.615801239013672
Epoch 4330 0.029346745419833394 8.62168092727661
Epoch 4331 0.029275527141160437 8.62207908630371
Epoch 4332 0.02937869005319145 8.623494720458984
Epoch 4333 0.029574912496738964 8.636596155166625
Epoch 4334 0.02941345243404309 8.632451820373536
Epoch 4335 0.02934232217570146 8.627283477783203
Epoch 4336 0.02968860591451327 8.623214960098267
Epoch 4337 0.029617900815274982 8.628540515899658
Epoch 4338 0.029305137486921415 8.633312034606934
Epoch 4339 0.029372248198423122 8.636498355865479
Epoch 4340 0.02934088415155808 8.636508464813232
Epoch 4341 0.02949589751660824 8.6330397605896
Epoch 4342 0.02950546582125955 8.63449649810791
Epoch 4343 0.029540154006746082 8.648384380340577
Epoch 4344 0.02990237068798807 8.639788818359374
Epoch 4345 0.02937744804140594 8.635269832611083
Epoch 4346 0.029723032067219417 8.6395854473114
Epoch 4347 0.029457231611013414 8.640587520599365
Epoch 4348 0.0296

Epoch 4458 0.02901214085933235 8.650586318969726
Epoch 4459 0.028946352005004884 8.64810562133789
Epoch 4460 0.02885147819502486 8.6474365234375
Epoch 4461 0.02902298480686214 8.642538928985596
Epoch 4462 0.028887384798791674 8.656278800964355
Epoch 4463 0.029044922958645556 8.652577590942382
Epoch 4464 0.02918148955537213 8.650642681121827
Epoch 4465 0.029065911223491035 8.645921611785889
Epoch 4466 0.02900791441400846 8.657538604736327
Epoch 4467 0.028979530785646704 8.656515502929688
Epoch 4468 0.028901776795585952 8.650046014785767
Epoch 4469 0.02904153244776858 8.658582735061646
Epoch 4470 0.028727555688884524 8.656681728363036
Epoch 4471 0.028704367246892716 8.657898664474487
Epoch 4472 0.028653240348729823 8.666394329071045
Epoch 4473 0.029104994299511116 8.654007053375244
Epoch 4474 0.028739392571151257 8.655403184890748
Epoch 4475 0.02891493828760253 8.650689125061035
Epoch 4476 0.028717754843334355 8.650680875778198
Epoch 4477 0.029239445303877194 8.654525709152221
Epoch 4478

Epoch 4600 0.028524549740056197 8.664960670471192
Time1:  0.2578737922012806
Time2:  3.9535233974456787
Time3:  15.82808637432754
Time1:  0.028924720361828804
Time2:  0.42919897101819515
Time3:  1.7489221654832363

train {'F1@10': 0.19237352907657623, 'F1@30': 0.4975413382053375, 'F1@50': 0.6970452666282654, 'ndcg@10': 0.4912356136565628, 'ndcg@30': 0.6287114135181331, 'ndcg@50': 0.7353134398482267, 'ndcg@all': 0.7335930843589966}
valid {'F1@10': 0.11290538311004639, 'F1@30': 0.1986413598060608, 'F1@50': 0.23032821714878082, 'ndcg@10': 0.34847512748465315, 'ndcg@30': 0.3444881222907219, 'ndcg@50': 0.36071296673998965, 'ndcg@all': 0.5591648723091092}
Epoch 4601 0.02862893758962552 8.671133947372436
Epoch 4602 0.028553549986746578 8.675738096237183
Epoch 4603 0.0286311874166131 8.670692014694215
Epoch 4604 0.028409454226493835 8.674978399276734
Epoch 4605 0.028557978239324357 8.67033805847168
Epoch 4606 0.028608204196724628 8.664687633514404
Epoch 4607 0.028582812514570025 8.680933475494

Epoch 4730 0.028026378610067897 8.694759082794189
Epoch 4731 0.02777672770122687 8.690771484375
Epoch 4732 0.02786417535195748 8.685422325134278
Epoch 4733 0.027732363177670374 8.694517040252686
Epoch 4734 0.02757557239383459 8.696991395950317
Epoch 4735 0.028157026258607706 8.69846978187561
Epoch 4736 0.027807343213094606 8.703439474105835
Epoch 4737 0.028072274724642434 8.69614224433899
Epoch 4738 0.027929879642195173 8.700318145751954
Epoch 4739 0.027998955527113543 8.682997512817384
Epoch 4740 0.027899964981608922 8.699979972839355
Epoch 4741 0.027925808500084612 8.685987281799317
Epoch 4742 0.027911795862019063 8.690841484069825
Epoch 4743 0.02792750605278545 8.69177770614624
Epoch 4744 0.02808500298609336 8.689690685272216
Epoch 4745 0.027796277797056568 8.703771591186523
Epoch 4746 0.02799513971226083 8.699297142028808
Epoch 4747 0.027834707270893787 8.701103210449219
Epoch 4748 0.02792649625076188 8.706213331222534
Epoch 4749 0.027984517916209168 8.695131015777587
Epoch 4750 0.

Epoch 4859 0.027414239301449723 8.719456911087036
Epoch 4860 0.027584816743102338 8.715330600738525
Epoch 4861 0.027480417241652805 8.729506683349609
Epoch 4862 0.027578793300522698 8.723277568817139
Epoch 4863 0.027550171626110873 8.717992305755615
Epoch 4864 0.02758892306851016 8.70523681640625
Epoch 4865 0.027740880598624548 8.710594749450683
Epoch 4866 0.027464465859035652 8.70513849258423
Epoch 4867 0.02743535985549291 8.708373975753783
Epoch 4868 0.02754152226779196 8.714540576934814
Epoch 4869 0.027433682212399113 8.714008045196532
Epoch 4870 0.02766302964753575 8.718526840209961
Epoch 4871 0.027396050799224112 8.720158433914184
Epoch 4872 0.027252088776893085 8.711903381347657
Epoch 4873 0.02737470469954941 8.725405883789062
Epoch 4874 0.027288570441305636 8.719518804550171
Epoch 4875 0.027437323269744716 8.714760494232177
Epoch 4876 0.027644457730154195 8.712922048568725
Epoch 4877 0.027287240905894173 8.721023988723754
Epoch 4878 0.02751271403912041 8.705244255065917
Epoch 48

In [55]:
def listNet(y_pred, y_true, eps=DEFAULT_EPS, padded_value_indicator=PADDED_Y_VALUE):
    """
    ListNet loss introduced in "Learning to Rank: From Pairwise Approach to Listwise Approach".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param eps: epsilon value, used for numerical stability
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: loss value, a torch.Tensor
    """
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    mask = y_true == padded_value_indicator
    y_pred[mask] = float('-inf')
    y_true[mask] = float('-inf')

    preds_smax = F.softmax(y_pred, dim=1)
    true_smax = F.softmax(y_true, dim=1)

    preds_smax = preds_smax + eps
    preds_log = torch.log(preds_smax)

    return torch.mean(-torch.sum(true_smax * preds_log, dim=1))

In [56]:
# setting
lr = 0.05
momentum = 0.
weight_decay = 0
nesterov = False # True

n_epoch = 5000
verbose = True
valid_epoch = 50

h_dim = 3000

model = MLPDecoder(doc_emb_dim=document_vectors.shape[1], num_words=len(word_vectors), h_dim=h_dim).to(device)
model.train()

word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)

opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
# opt = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.MSELoss(reduction='mean')
criterion = nn.MultiLabelMarginLoss(reduction='mean')
# criterion = nn.MultiLabelSoftMarginLoss(reduction='mean')

results = []
step = 0
clip_value = 1

for epoch in tqdm(range(n_epoch)):    
    train_loss_his = []
    valid_loss_his = []
    
    model.train()

    for data in train_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
#         loss = criterion(pred, target)
#         loss = criterion(pred, target_rank)
#         loss = criterion(pred, torch.sign(target))
        loss = listNet(pred, target)
    
        # Model backwarding
        model.zero_grad()
        loss.backward()
#         torch.nn.utils.clip_grad_value_(model.parameters(), clip_value)
        
        opt.step()

        train_loss_his.append(loss.item())
        
    model.eval()
    for data in valid_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
        
#         loss = criterion(pred, target)
#         loss = criterion(pred, target_rank)
#         loss = criterion(pred, torch.sign(target))
        loss = listNet(pred, target)
    
        valid_loss_his.append(loss.item())
    
    print("Epoch", epoch, np.mean(train_loss_his), np.mean(valid_loss_his))
    
    if epoch % valid_epoch == 0:
        res = {}
        res['epoch'] = epoch
        
        train_res_ndcg = evaluate_MLPDecoder(model, train_loader)
        valid_res_ndcg = evaluate_MLPDecoder(model, valid_loader)
        
        res.update(valid_res_ndcg)
        results.append(res)
        
        if verbose:
            print()
            print('train', train_res_ndcg)
            print('valid', valid_res_ndcg)
            


  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0 8.282003551059299 7.740373373031616
Time1:  0.25538268126547337
Time2:  3.970807919278741
Time3:  15.84010280482471
Time1:  0.028463536873459816
Time2:  0.4305055998265743
Time3:  1.750818321481347

train {'F1@10': 0.04074257239699364, 'F1@30': 0.05158373713493347, 'F1@50': 0.05294894054532051, 'ndcg@10': 0.14633899610564713, 'ndcg@30': 0.1295833511801967, 'ndcg@50': 0.12889668041308233, 'ndcg@all': 0.35752222899500863}
valid {'F1@10': 0.038714658468961716, 'F1@30': 0.049900904297828674, 'F1@50': 0.05214732512831688, 'ndcg@10': 0.13092860741703002, 'ndcg@30': 0.11550609224724832, 'ndcg@50': 0.11484438787867969, 'ndcg@all': 0.3468204210172067}
Epoch 1 6.985731659995185 7.194931888580323
Epoch 2 6.092027176751031 6.780399131774902
Epoch 3 5.346906359990438 6.456426000595092
Epoch 4 4.7333599037594265 6.172033596038818
Epoch 5 4.207616909344991 5.953709936141967
Epoch 6 3.7671866178512574 5.7682637691497805
Epoch 7 3.3899072064293754 5.619031810760498
Epoch 8 3.0724344041612413 5.

Epoch 139 1.263231109910541 4.2111776113510135
Epoch 140 1.2632724603017171 4.2101717710495
Epoch 141 1.2623099015818702 4.2047199010849
Epoch 142 1.2624440020985073 4.201068496704101
Epoch 143 1.2615688337220086 4.201591038703919
Epoch 144 1.2610105521149106 4.203146052360535
Epoch 145 1.2602436754438613 4.199316692352295
Epoch 146 1.260306899415122 4.195049381256103
Epoch 147 1.259492959578832 4.19432532787323
Epoch 148 1.2589012689060635 4.19471378326416
Epoch 149 1.2583658854166666 4.193408131599426
Epoch 150 1.2579590035809411 4.194846963882446
Time1:  0.2554880306124687
Time2:  3.967317884787917
Time3:  15.88158598728478
Time1:  0.028435220941901207
Time2:  0.43453568033874035
Time3:  1.756269283592701

train {'F1@10': 0.13561570644378662, 'F1@30': 0.15707826614379883, 'F1@50': 0.15281274914741516, 'ndcg@10': 0.6535564544990874, 'ndcg@30': 0.5359639566180372, 'ndcg@50': 0.5172786758932906, 'ndcg@all': 0.6923878856865198}
valid {'F1@10': 0.11341048032045364, 'F1@30': 0.14147754013

Epoch 276 1.2259035262796614 4.0658392906188965
Epoch 277 1.2257415937052833 4.068649816513061
Epoch 278 1.2255081944995456 4.069665169715881
Epoch 279 1.2253522766960991 4.068708968162537
Epoch 280 1.2256091521845924 4.0657330513000485
Epoch 281 1.2250961131519742 4.064980101585388
Epoch 282 1.224692157904307 4.06555917263031
Epoch 283 1.224692157904307 4.062156391143799
Epoch 284 1.2246405495537651 4.061219263076782
Epoch 285 1.2247793310218387 4.062985777854919
Epoch 286 1.2244803845882415 4.062053465843201
Epoch 287 1.2240287376774681 4.061098313331604
Epoch 288 1.2242251879639097 4.062237739562988
Epoch 289 1.2239112436771393 4.062393832206726
Epoch 290 1.2244316624270546 4.05859341621399
Epoch 291 1.2241053091155158 4.059314131736755
Epoch 292 1.223591085937288 4.059266829490662
Epoch 293 1.2239060408539242 4.058769297599793
Epoch 294 1.2229907121923236 4.058314251899719
Epoch 295 1.2233721243010627 4.057916903495789
Epoch 296 1.222854165898429 4.058439946174621
Epoch 297 1.22348

Epoch 413 1.2119596653514437 4.007680416107178
Epoch 414 1.211221135324902 4.0049227476119995
Epoch 415 1.211825070116255 4.0024110078811646
Epoch 416 1.2121552871333228 4.003641796112061
Epoch 417 1.2105319877465566 4.000801849365234
Epoch 418 1.2110837890042199 4.00432984828949
Epoch 419 1.211373828517066 4.001468443870545
Epoch 420 1.2111894508202872 4.004180073738098
Epoch 421 1.2109325302971734 4.0048997640609745
Epoch 422 1.211112129688263 4.0002758502960205
Epoch 423 1.2112083103921678 4.001972627639771
Epoch 424 1.2104515486293368 3.9997998237609864
Epoch 425 1.2113604254192776 3.9991337776184084
Epoch 426 1.2104715764522553 4.00017204284668
Epoch 427 1.2112419989373948 3.997720813751221
Epoch 428 1.2104423112339444 4.002051377296448
Epoch 429 1.2106835775905185 3.9994035720825196
Epoch 430 1.2108005550172594 3.998451328277588
Epoch 431 1.2106362018320296 3.9969583988189696
Epoch 432 1.210180424981647 4.000105857849121
Epoch 433 1.2094403598043653 3.9973737478256224
Epoch 434 1

...

Epoch 4401 1.1691047569115958 3.96573326587677
Epoch 4402 1.16841083433893 3.961131954193115
Epoch 4403 1.1684832784864638 3.9566322803497314
Epoch 4404 1.1687785135375128 3.9646331071853638
Epoch 4405 1.1690763884120516 3.9626843214035032
Epoch 4406 1.1682941986454858 3.9619353771209718
Epoch 4407 1.1688477026091681 3.9579038619995117
Epoch 4408 1.1688388956917657 3.9613857746124266
Epoch 4409 1.1686950471666124 3.9578627824783323
Epoch 4410 1.1684579968452453 3.961476182937622
Epoch 4411 1.168502637412813 3.9594822406768797
Epoch 4412 1.1684431778060065 3.95615496635437
Epoch 4413 1.1689589947462082 3.9591180801391603
Epoch 4414 1.168913949198193 3.955283784866333
Epoch 4415 1.1681882341702778 3.9614351272583006
Epoch 4416 1.168787278731664 3.9598438262939455
Epoch 4417 1.1685082342889574 3.9612508535385134
Epoch 4418 1.1690841999318864 3.9630778789520265
Epoch 4419 1.1682618313365511 3.9595579147338866
Epoch 4420 1.16870137585534 3.9590271949768066
Epoch 4421 1.1680792027049594 3.96

Epoch 4546 1.168256500032213 3.966037940979004
Epoch 4547 1.1691234469413758 3.9696967601776123
Epoch 4548 1.168587139579985 3.9658596754074096
Epoch 4549 1.1688888768355052 3.965130829811096
Epoch 4550 1.1682601597574023 3.967977738380432
Time1:  0.25571646727621555
Time2:  3.9625871665775776
Time3:  15.863826112821698
Time1:  0.02847956493496895
Time2:  0.43031965009868145
Time3:  1.7510002367198467

train {'F1@10': 0.185836061835289, 'F1@30': 0.2282114326953888, 'F1@50': 0.21890656650066376, 'ndcg@10': 0.7749225826165302, 'ndcg@30': 0.6519925903425333, 'ndcg@50': 0.630718545514564, 'ndcg@all': 0.7760015363654783}
valid {'F1@10': 0.13370373845100403, 'F1@30': 0.17341597378253937, 'F1@50': 0.17407920956611633, 'ndcg@10': 0.5357104120226027, 'ndcg@30': 0.46488717801727214, 'ndcg@50': 0.455400214173474, 'ndcg@all': 0.6375818651866865}
Epoch 4551 1.168306589126587 3.965852379798889
Epoch 4552 1.1684383816189237 3.96849410533905
Epoch 4553 1.1684189882543352 3.963755464553833
Epoch 4554 1

Epoch 4679 1.1674600395891401 3.9719737768173218
Epoch 4680 1.168486777941386 3.9743817806243897
Epoch 4681 1.1676471418804593 3.9703279972076415
Epoch 4682 1.1671780678961012 3.972154426574707
Epoch 4683 1.1674615813626184 3.9677141904830933
Epoch 4684 1.1677422953976526 3.965385580062866
Epoch 4685 1.16770840883255 3.9724135637283324
Epoch 4686 1.1680107911427815 3.9697240591049194
Epoch 4687 1.1680881261825562 3.9695950984954833
Epoch 4688 1.1680275354120466 3.9665042400360107
Epoch 4689 1.1681327905919816 3.9671592235565187
Epoch 4690 1.1676384978824192 3.9654650688171387
Epoch 4691 1.167839056915707 3.9671314001083373
Epoch 4692 1.1682360576258766 3.9685758352279663
Epoch 4693 1.1679765204588572 3.9708054304122924
Epoch 4694 1.168201955821779 3.969405937194824
Epoch 4695 1.1684908833768632 3.9696892499923706
Epoch 4696 1.1680777960353428 3.970832920074463
Epoch 4697 1.168077954981062 3.9657038927078245
Epoch 4698 1.1679920607142977 3.9653351306915283
Epoch 4699 1.1679037272930146 

Epoch 4812 1.1672775977187686 3.9747653007507324
Epoch 4813 1.1672821323076883 3.974014711380005
Epoch 4814 1.1676717930369906 3.9737016439437864
Epoch 4815 1.167243053515752 3.9738086462020874
Epoch 4816 1.1675302777025434 3.974694561958313
Epoch 4817 1.1680070320765177 3.972719407081604
Epoch 4818 1.1677178064982097 3.9736797332763674
Epoch 4819 1.1675103578302595 3.9727112293243407
Epoch 4820 1.1678763965765635 3.973256325721741
Epoch 4821 1.167923284901513 3.97327606678009
Epoch 4822 1.1677608847618104 3.9712779045104982
Epoch 4823 1.1672236601511636 3.973597836494446
Epoch 4824 1.1675588952170477 3.9737276077270507
Epoch 4825 1.1675576355722215 3.970454382896423
Epoch 4826 1.1679449101289114 3.971514105796814
Epoch 4827 1.1671906027528975 3.9711604833602907
Epoch 4828 1.1674522373411391 3.976252579689026
Epoch 4829 1.1671211196316613 3.9772651195526123
Epoch 4830 1.1671992242336273 3.9733057022094727
Epoch 4831 1.167876101202435 3.975286865234375
Epoch 4832 1.1674926857153574 3.97

Epoch 4951 1.1668488535616133 3.9790252447128296
Epoch 4952 1.1670742372671763 3.9761707067489622
Epoch 4953 1.1677469359503851 3.9764343976974486
Epoch 4954 1.1672996587223476 3.978725552558899
Epoch 4955 1.167238908343845 3.9816753387451174
Epoch 4956 1.166672670841217 3.979105520248413
Epoch 4957 1.1674071106645796 3.9818819761276245
Epoch 4958 1.1673452231619093 3.9802659511566163
Epoch 4959 1.1673103166951073 3.9779738187789917
Epoch 4960 1.1670275469621023 3.980030560493469
Epoch 4961 1.1675269895129734 3.977017068862915
Epoch 4962 1.1673874759011797 3.977394437789917
Epoch 4963 1.1672039555178748 3.9769142866134644
Epoch 4964 1.167130868964725 3.976837229728699
Epoch 4965 1.1671762062443627 3.979563069343567
Epoch 4966 1.1672773917516073 3.977637457847595
Epoch 4967 1.1677186787128448 3.976111364364624
Epoch 4968 1.1666782206959194 3.9795403242111207
Epoch 4969 1.166677176952362 3.979121994972229
Epoch 4970 1.166628101799223 3.977053904533386
Epoch 4971 1.167515348063575 3.98111

In [ ]:
def deterministic_neural_sort(s, tau, mask):
    """
    Deterministic neural sort.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param s: values to sort, shape [batch_size, slate_length]
    :param tau: temperature for the final softmax function
    :param mask: mask indicating padded elements
    :return: approximate permutation matrices of shape [batch_size, slate_length, slate_length]
    """
    dev = get_torch_device()

    n = s.size()[1]
    one = torch.ones((n, 1), dtype=torch.float32, device=dev)
    s = s.masked_fill(mask[:, :, None], -1e8)
    A_s = torch.abs(s - s.permute(0, 2, 1))
    A_s = A_s.masked_fill(mask[:, :, None] | mask[:, None, :], 0.0)

    B = torch.matmul(A_s, torch.matmul(one, torch.transpose(one, 0, 1)))

    temp = [n - m + 1 - 2 * (torch.arange(n - m, device=dev) + 1) for m in mask.squeeze(-1).sum(dim=1)]
    temp = [t.type(torch.float32) for t in temp]
    temp = [torch.cat((t, torch.zeros(n - len(t), device=dev))) for t in temp]
    scaling = torch.stack(temp).type(torch.float32).to(dev)  # type: ignore

    s = s.masked_fill(mask[:, :, None], 0.0)
    C = torch.matmul(s, scaling.unsqueeze(-2))

    P_max = (C - B).permute(0, 2, 1)
    P_max = P_max.masked_fill(mask[:, :, None] | mask[:, None, :], -np.inf)
    P_max = P_max.masked_fill(mask[:, :, None] & mask[:, None, :], 1.0)
    sm = torch.nn.Softmax(-1)
    P_hat = sm(P_max / tau)
    return P_hat

def sinkhorn_scaling(mat, mask=None, tol=1e-6, max_iter=50):
    """
    Sinkhorn scaling procedure.
    :param mat: a tensor of square matrices of shape N x M x M, where N is batch size
    :param mask: a tensor of masks of shape N x M
    :param tol: Sinkhorn scaling tolerance
    :param max_iter: maximum number of iterations of the Sinkhorn scaling
    :return: a tensor of (approximately) doubly stochastic matrices
    """
    if mask is not None:
        mat = mat.masked_fill(mask[:, None, :] | mask[:, :, None], 0.0)
        mat = mat.masked_fill(mask[:, None, :] & mask[:, :, None], 1.0)

    for _ in range(max_iter):
        mat = mat / mat.sum(dim=1, keepdim=True).clamp(min=DEFAULT_EPS)
        mat = mat / mat.sum(dim=2, keepdim=True).clamp(min=DEFAULT_EPS)

        if torch.max(torch.abs(mat.sum(dim=2) - 1.)) < tol and torch.max(torch.abs(mat.sum(dim=1) - 1.)) < tol:
            break

    if mask is not None:
        mat = mat.masked_fill(mask[:, None, :] | mask[:, :, None], 0.0)

    return mat

def stochastic_neural_sort(s, n_samples, tau, mask, beta=1.0, log_scores=True, eps=1e-10):
    """
    Stochastic neural sort. Please note that memory complexity grows by factor n_samples.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param s: values to sort, shape [batch_size, slate_length]
    :param n_samples: number of samples (approximations) for each permutation matrix
    :param tau: temperature for the final softmax function
    :param mask: mask indicating padded elements
    :param beta: scale parameter for the Gumbel distribution
    :param log_scores: whether to apply the logarithm function to scores prior to Gumbel perturbation
    :param eps: epsilon for the logarithm function
    :return: approximate permutation matrices of shape [n_samples, batch_size, slate_length, slate_length]
    """
    dev = get_torch_device()

    batch_size = s.size()[0]
    n = s.size()[1]
    s_positive = s + torch.abs(s.min())
    samples = beta * sample_gumbel([n_samples, batch_size, n, 1], device=dev)
    if log_scores:
        s_positive = torch.log(s_positive + eps)

    s_perturb = (s_positive + samples).view(n_samples * batch_size, n, 1)
    mask_repeated = mask.repeat_interleave(n_samples, dim=0)

    P_hat = deterministic_neural_sort(s_perturb, tau, mask_repeated)
    P_hat = P_hat.view(n_samples, batch_size, n, n)
    return P_hat

def dcg(y_pred, y_true, ats=None, gain_function=lambda x: torch.pow(2, x) - 1, padding_indicator=PADDED_Y_VALUE):
    """
    Discounted Cumulative Gain at k.
    Compute DCG at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for DCG evaluation, if None, maximum rank is used
    :param gain_function: callable, gain function for the ground truth labels, e.g. torch.pow(2, x) - 1
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: DCG values for each slate and evaluation position, shape [batch_size, len(ats)]
    """
    y_true = y_true.clone()
    y_pred = y_pred.clone()

    actual_length = y_true.shape[1]

    if ats is None:
        ats = [actual_length]
    ats = [min(at, actual_length) for at in ats]

    true_sorted_by_preds = __apply_mask_and_get_true_sorted_by_preds(y_pred, y_true, padding_indicator)

    discounts = (torch.tensor(1) / torch.log2(torch.arange(true_sorted_by_preds.shape[1], dtype=torch.float) + 2.0)).to(
        device=true_sorted_by_preds.device)

    gains = gain_function(true_sorted_by_preds)

    discounted_gains = (gains * discounts)[:, :np.max(ats)]

    cum_dcg = torch.cumsum(discounted_gains, dim=1)

    ats_tensor = torch.tensor(ats, dtype=torch.long) - torch.tensor(1)

    dcg = cum_dcg[:, ats_tensor]

    return dcg

def get_torch_device():
    """
    Getter for an available pyTorch device.
    :return: CUDA-capable GPU if available, CPU otherwise
    """
    return "cpu"
    return torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")


def neuralNDCG(y_pred, y_true, padded_value_indicator=PADDED_Y_VALUE, temperature=1., powered_relevancies=True, k=None,
               stochastic=False, n_samples=32, beta=0.1, log_scores=True):
    """
    NeuralNDCG loss introduced in "NeuralNDCG: Direct Optimisation of a Ranking Metric via Differentiable
    Relaxation of Sorting" - https://arxiv.org/abs/2102.07831. Based on the NeuralSort algorithm.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param temperature: temperature for the NeuralSort algorithm
    :param powered_relevancies: whether to apply 2^x - 1 gain function, x otherwise
    :param k: rank at which the loss is truncated
    :param stochastic: whether to calculate the stochastic variant
    :param n_samples: how many stochastic samples are taken, used if stochastic == True
    :param beta: beta parameter for NeuralSort algorithm, used if stochastic == True
    :param log_scores: log_scores parameter for NeuralSort algorithm, used if stochastic == True
    :return: loss value, a torch.Tensor
    """
    dev = get_torch_device()

    if k is None:
        k = y_true.shape[1]

    mask = (y_true == padded_value_indicator)
    # Choose the deterministic/stochastic variant
    if stochastic:
        P_hat = stochastic_neural_sort(y_pred.unsqueeze(-1), n_samples=n_samples, tau=temperature, mask=mask,
                                       beta=beta, log_scores=log_scores)
    else:
        P_hat = deterministic_neural_sort(y_pred.unsqueeze(-1), tau=temperature, mask=mask).unsqueeze(0)

    # Perform sinkhorn scaling to obtain doubly stochastic permutation matrices
    P_hat = sinkhorn_scaling(P_hat.view(P_hat.shape[0] * P_hat.shape[1], P_hat.shape[2], P_hat.shape[3]),
                             mask.repeat_interleave(P_hat.shape[0], dim=0), tol=1e-6, max_iter=50)
    P_hat = P_hat.view(int(P_hat.shape[0] / y_pred.shape[0]), y_pred.shape[0], P_hat.shape[1], P_hat.shape[2])

    # Mask P_hat and apply to true labels, ie approximately sort them
    P_hat = P_hat.masked_fill(mask[None, :, :, None] | mask[None, :, None, :], 0.)
    y_true_masked = y_true.masked_fill(mask, 0.).unsqueeze(-1).unsqueeze(0)
    if powered_relevancies:
        y_true_masked = torch.pow(2., y_true_masked) - 1.

    ground_truth = torch.matmul(P_hat, y_true_masked).squeeze(-1)
    discounts = (torch.tensor(1.) / torch.log2(torch.arange(y_true.shape[-1], dtype=torch.float) + 2.)).to(dev)
    discounted_gains = ground_truth * discounts

    if powered_relevancies:
        idcg = dcg(y_true, y_true, ats=[k]).permute(1, 0)
    else:
        idcg = dcg(y_true, y_true, ats=[k], gain_function=lambda x: x).permute(1, 0)

    discounted_gains = discounted_gains[:, :, :k]
    ndcg = discounted_gains.sum(dim=-1) / (idcg + DEFAULT_EPS)
    idcg_mask = idcg == 0.
    ndcg = ndcg.masked_fill(idcg_mask.repeat(ndcg.shape[0], 1), 0.)

    assert (ndcg < 0.).sum() >= 0, "every ndcg should be non-negative"
    if idcg_mask.all():
        return torch.tensor(0.)

    mean_ndcg = ndcg.sum() / ((~idcg_mask).sum() * ndcg.shape[0])  # type: ignore
    return -1. * mean_ndcg  # -1 cause we want to maximize NDCG


def neuralNDCG_transposed(y_pred, y_true, padded_value_indicator=PADDED_Y_VALUE, temperature=1.,
                          powered_relevancies=True, k=None, stochastic=False, n_samples=32, beta=0.1, log_scores=True,
                          max_iter=50, tol=1e-6):
    """
    NeuralNDCG Transposed loss introduced in "NeuralNDCG: Direct Optimisation of a Ranking Metric via Differentiable
    Relaxation of Sorting" - https://arxiv.org/abs/2102.07831. Based on the NeuralSort algorithm.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param temperature: temperature for the NeuralSort algorithm
    :param powered_relevancies: whether to apply 2^x - 1 gain function, x otherwise
    :param k: rank at which the loss is truncated
    :param stochastic: whether to calculate the stochastic variant
    :param n_samples: how many stochastic samples are taken, used if stochastic == True
    :param beta: beta parameter for NeuralSort algorithm, used if stochastic == True
    :param log_scores: log_scores parameter for NeuralSort algorithm, used if stochastic == True
    :param max_iter: maximum iteration count for Sinkhorn scaling
    :param tol: tolerance for Sinkhorn scaling
    :return: loss value, a torch.Tensor
    """
    dev = get_torch_device()

    if k is None:
        k = y_true.shape[1]

    mask = (y_true == padded_value_indicator)

    if stochastic:
        P_hat = stochastic_neural_sort(y_pred.unsqueeze(-1), n_samples=n_samples, tau=temperature, mask=mask,
                                       beta=beta, log_scores=log_scores)
    else:
        P_hat = deterministic_neural_sort(y_pred.unsqueeze(-1), tau=temperature, mask=mask).unsqueeze(0)

    # Perform sinkhorn scaling to obtain doubly stochastic permutation matrices
    P_hat_masked = sinkhorn_scaling(P_hat.view(P_hat.shape[0] * y_pred.shape[0], y_pred.shape[1], y_pred.shape[1]),
                                    mask.repeat_interleave(P_hat.shape[0], dim=0), tol=tol, max_iter=max_iter)
    P_hat_masked = P_hat_masked.view(P_hat.shape[0], y_pred.shape[0], y_pred.shape[1], y_pred.shape[1])
    discounts = (torch.tensor(1) / torch.log2(torch.arange(y_true.shape[-1], dtype=torch.float) + 2.)).to(dev)

    # This takes care of the @k metric truncation - if something is @>k, it is useless and gets 0.0 discount
    discounts[k:] = 0.
    discounts = discounts[None, None, :, None]

    # Here the discounts become expected discounts
    discounts = torch.matmul(P_hat_masked.permute(0, 1, 3, 2), discounts).squeeze(-1)
    if powered_relevancies:
        gains = torch.pow(2., y_true) - 1
        discounted_gains = gains.unsqueeze(0) * discounts
        idcg = dcg(y_true, y_true, ats=[k]).squeeze()
    else:
        gains = y_true
        discounted_gains = gains.unsqueeze(0) * discounts
        idcg = dcg(y_true, y_true, ats=[k]).squeeze()

    ndcg = discounted_gains.sum(dim=2) / (idcg + DEFAULT_EPS)
    idcg_mask = idcg == 0.
    ndcg = ndcg.masked_fill(idcg_mask, 0.)

    assert (ndcg < 0.).sum() >= 0, "every ndcg should be non-negative"
    if idcg_mask.all():
        return torch.tensor(0.)

    mean_ndcg = ndcg.sum() / ((~idcg_mask).sum() * ndcg.shape[0])  # type: ignore
    return -1. * mean_ndcg  # -1 cause we want to maximize NDCG

In [ ]:
def lambdaLoss(y_pred, y_true, eps=DEFAULT_EPS, padded_value_indicator=PADDED_Y_VALUE, weighing_scheme=None, k=None, sigma=1., mu=10.,
               reduction="sum", reduction_log="binary"):
    """
    LambdaLoss framework for LTR losses implementations, introduced in "The LambdaLoss Framework for Ranking Metric Optimization".
    Contains implementations of different weighing schemes corresponding to e.g. LambdaRank or RankNet.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param eps: epsilon value, used for numerical stability
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param weighing_scheme: a string corresponding to a name of one of the weighing schemes
    :param k: rank at which the loss is truncated
    :param sigma: score difference weight used in the sigmoid function
    :param mu: optional weight used in NDCGLoss2++ weighing scheme
    :param reduction: losses reduction method, could be either a sum or a mean
    :param reduction_log: logarithm variant used prior to masking and loss reduction, either binary or natural
    :return: loss value, a torch.Tensor
    """
    device = y_pred.device
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    padded_mask = y_true == padded_value_indicator
    y_pred[padded_mask] = float("-inf")
    y_true[padded_mask] = float("-inf")

    # Here we sort the true and predicted relevancy scores.
    y_pred_sorted, indices_pred = y_pred.sort(descending=True, dim=-1)
    y_true_sorted, _ = y_true.sort(descending=True, dim=-1)

    # After sorting, we can mask out the pairs of indices (i, j) containing index of a padded element.
    true_sorted_by_preds = torch.gather(y_true, dim=1, index=indices_pred)
    true_diffs = true_sorted_by_preds[:, :, None] - true_sorted_by_preds[:, None, :]
    print(true_sorted_by_preds.shape)
    print(true_diffs.shape)
    padded_pairs_mask = torch.isfinite(true_diffs)

    if weighing_scheme != "ndcgLoss1_scheme":
        padded_pairs_mask = padded_pairs_mask & (true_diffs > 0)

    ndcg_at_k_mask = torch.zeros((y_pred.shape[1], y_pred.shape[1]), dtype=torch.bool, device=device)
    ndcg_at_k_mask[:k, :k] = 1

    # Here we clamp the -infs to get correct gains and ideal DCGs (maxDCGs)
    true_sorted_by_preds.clamp_(min=0.)
    y_true_sorted.clamp_(min=0.)

    # Here we find the gains, discounts and ideal DCGs per slate.
    pos_idxs = torch.arange(1, y_pred.shape[1] + 1).to(device)
    D = torch.log2(1. + pos_idxs.float())[None, :]
    maxDCGs = torch.sum(((torch.pow(2, y_true_sorted) - 1) / D)[:, :k], dim=-1).clamp(min=eps)
    G = (torch.pow(2, true_sorted_by_preds) - 1) / maxDCGs[:, None]

    # Here we apply appropriate weighing scheme - ndcgLoss1, ndcgLoss2, ndcgLoss2++ or no weights (=1.0)
    if weighing_scheme is None:
        weights = 1.
    else:
        weights = globals()[weighing_scheme](G, D, mu, true_sorted_by_preds)  # type: ignore

    # We are clamping the array entries to maintain correct backprop (log(0) and division by 0)
    scores_diffs = (y_pred_sorted[:, :, None] - y_pred_sorted[:, None, :]).clamp(min=-1e8, max=1e8)
    scores_diffs.masked_fill(torch.isnan(scores_diffs), 0.)
    weighted_probas = (torch.sigmoid(sigma * scores_diffs).clamp(min=eps) ** weights).clamp(min=eps)
    if reduction_log == "natural":
        losses = torch.log(weighted_probas)
    elif reduction_log == "binary":
        losses = torch.log2(weighted_probas)
    else:
        raise ValueError("Reduction logarithm base can be either natural or binary")

    if reduction == "sum":
        loss = -torch.sum(losses[padded_pairs_mask & ndcg_at_k_mask])
    elif reduction == "mean":
        loss = -torch.mean(losses[padded_pairs_mask & ndcg_at_k_mask])
    else:
        raise ValueError("Reduction method can be either sum or mean")

    return loss


def ndcgLoss1_scheme(G, D, *args):
    return (G / D)[:, :, None]


def ndcgLoss2_scheme(G, D, *args):
    pos_idxs = torch.arange(1, G.shape[1] + 1, device=G.device)
    delta_idxs = torch.abs(pos_idxs[:, None] - pos_idxs[None, :])
    deltas = torch.abs(torch.pow(torch.abs(D[0, delta_idxs - 1]), -1.) - torch.pow(torch.abs(D[0, delta_idxs]), -1.))
    deltas.diagonal().zero_()

    return deltas[None, :, :] * torch.abs(G[:, :, None] - G[:, None, :])


def lambdaRank_scheme(G, D, *args):
    return torch.abs(torch.pow(D[:, :, None], -1.) - torch.pow(D[:, None, :], -1.)) * torch.abs(G[:, :, None] - G[:, None, :])


def ndcgLoss2PP_scheme(G, D, *args):
    return args[0] * ndcgLoss2_scheme(G, D) + lambdaRank_scheme(G, D)


def rankNet_scheme(G, D, *args):
    return 1.


def rankNetWeightedByGTDiff_scheme(G, D, *args):
    return torch.abs(args[1][:, :, None] - args[1][:, None, :])


def rankNetWeightedByGTDiffPowed_scheme(G, D, *args):
    return torch.abs(torch.pow(args[1][:, :, None], 2) - torch.pow(args[1][:, None, :], 2))

In [ ]:
# from torchmetrics.functional import retrieval_normalized_dcg
# preds = torch.tensor([[.1, .2, .3, 4, 70], [.1, .2, .3, 4, 2]])
# target = torch.tensor([[10, 0, 8, 1, 5], [10, 0, 0, 1, 5]])
# # preds = torch.tensor([[.1, .2, .3, 4, 70]])
# # target = torch.tensor([[10, 0, 0, 1, 6]])

# print(retrieval_normalized_dcg(preds[0], target[0], k=3))
# print(ndcg_score(target[0].reshape(1,-1), preds[0].reshape(1,-1), k=3))
# print(retrieval_normalized_dcg(preds[1], target[1], k=3))
# print(ndcg_score(target[1].reshape(1,-1), preds[1].reshape(1,-1), k=3))
# print(retrieval_normalized_dcg(preds, target, k=3))
# print(ndcg_score(target, preds, k=3))

# ndcg = RetrievalNormalizedDCG(k=3)
# ndcg(preds, target, indexes=torch.vstack([torch.arange(preds.shape[0]) for _ in range(preds.shape[1])]).T)